In [1]:
!pip -q install transformers torch numpy ftfy gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 96.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.9/140.9 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 KB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 36.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 KB 6.1 MB/s eta 0:00:00


In [2]:
!wget https://huggingface.co/spaces/DrishtiSharma/Text-to-Image-search-using-CLIP/raw/main/photos_debug.tsv000
!wget https://huggingface.co/spaces/DrishtiSharma/Text-to-Image-search-using-CLIP/resolve/main/features_debug.npy
!wget https://huggingface.co/spaces/DrishtiSharma/Text-to-Image-search-using-CLIP/raw/main/photo_ids_debug.csv

--2023-03-13 08:54:05--  https://huggingface.co/spaces/DrishtiSharma/Text-to-Image-search-using-CLIP/raw/main/photos_debug.tsv000
Resolving huggingface.co (huggingface.co)... 52.87.110.41, 52.206.41.232, 52.2.178.255, ...
Connecting to huggingface.co (huggingface.co)|52.87.110.41|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9070632 (8.7M) [text/plain]
Saving to: ‘photos_debug.tsv000’

photos_debug.tsv000 100%[===================>]   8.65M  47.0MB/s    in 0.2s    

2023-03-13 08:54:06 (47.0 MB/s) - ‘photos_debug.tsv000’ saved [9070632/9070632]

--2023-03-13 08:54:06--  https://huggingface.co/spaces/DrishtiSharma/Text-to-Image-search-using-CLIP/resolve/main/features_debug.npy
Resolving huggingface.co (huggingface.co)... 52.87.110.41, 52.206.41.232, 52.2.178.255, ...
Connecting to huggingface.co (huggingface.co)|52.87.110.41|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/spaces/DrishtiSharma/Text-t

In [3]:
#Acknowledgments:
#This project is inspired by:
#1. https://github.com/haltakov/natural-language-image-search by Vladimir Haltakov
#2. OpenAI's CLIP

#Import all the necessary libraries
import torch
import requests
import numpy as np
import pandas as pd
import gradio as gr
from io import BytesIO
from PIL import Image as PILIMAGE
from transformers import CLIPProcessor, CLIPModel, CLIPTokenizer

#Selecting device based on availability of GPUs
device = "cuda" if torch.cuda.is_available() else "cpu"
    
#Defining model, processor and tokenizer
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")

In [12]:
#Loading the data
photos = pd.read_csv("photos_debug.tsv000", sep='\t', header=0)
photo_features = np.load("features_debug.npy")
photo_ids = pd.read_csv("photo_ids_debug.csv")
photo_ids = list(photo_ids['photo_id'])
    
def find_best_matches(text):
    
    #Inference
    with torch.no_grad():
        # Encode and normalize the description using CLIP
        inputs = tokenizer([text],  padding=True, return_tensors="pt")
        inputs = processor(text=[text], images=None, return_tensors="pt", padding=True)
        
        # move inputs to gpu
        for k, v in inputs.items():
            inputs[k] = v.to(device)

        text_encoded =  model.get_text_features(**inputs).detach().cpu().numpy()
  
    
    # Finding Cosine similarity
    similarities = list((text_encoded @ photo_features.T).squeeze(0))
    
    #Block of code for displaying top 3 best matches (images)
    matched_images = []
    for i in range(3):
      idx = sorted(zip(similarities, range(photo_features.shape[0])), key=lambda x: x[0], reverse=True)[i][1]
      photo_id = photo_ids[idx]
      photo_data = photos[photos["photo_id"] == photo_id].iloc[0]
      response = requests.get(photo_data["photo_image_url"] + "?w=640")
      img = PILIMAGE.open(BytesIO(response.content))
      matched_images.append(img)

    return matched_images
    
    
#Gradio app     
iface = gr.Interface(
    fn=find_best_matches,
    inputs=[gr.inputs.Textbox(lines=1, label="Text query", placeholder="Introduce the search text...",)],
    examples=[["Dog sticking its tongue out"],["Traffic light on the right"],["Honey bee eating honey"],["Leaves of Bryophyllum fallen on the ground"], ["Cute Kangaroo"], ["Athlete holding a bike in his hands"], ["Happy puppy"], ["Sad puppy"], ["Leopard hiding in the bushes"]],
    theme = "grass",
    outputs = gr.Gallery(label="Generated images", show_label=False, elem_id="gallery"),
    enable_queue=True,
    title= "Text to Image search using CLIP",
    description="This application displays TOP THREE images from Unsplash dataset that best match the natural language search query provided by the user."
).launch(debug=True)

/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:40: UserWarning: `enable_queue` is deprecated in `Interface()`, please use it within `launch()` instead.
  warnings.warn(value)
/usr/local/lib/python3.9/dist-packages/gradio/interface.py:312: UserWarning: Currently, only the 'default' theme is supported.
  warnings.warn("Currently, only the 'default' theme is supported.")


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.
